In [16]:
import json

def clear_json(text):
    # result = text.replace("\n", ", ")
    result = text.replace("json", "")
    result = result.replace("`", "")
    result = result.replace("{,  \"","{\"") 
    result = result.replace(", }", "}")
    result = result.replace(",,", ",")
    result = result.strip(", ")
    result = result.strip()
    return result

with open("output/batch_jobs.json", "r") as f:
    jobs = json.load(f)

results = {}
input_token = 0
output_token = 0

L = len("/home/youyuan/Transcreation/Iris/download_images/")

for i in range(len(jobs)):
    
    results_file = f"output/batch_results_{i+1}.jsonl"
    record_file = f"output/batch_record_{i + 1}.json"
    
    with open(record_file, "r", encoding="utf-8") as f:
        r = json.load(f)
    with open(results_file, "r", encoding="utf-8") as f:
        for line in f:
            result = json.loads(line.strip())
            custom_id = result.get("custom_id", "")
            if custom_id not in r:
                print(f"Custom ID {result['custom_id']} not found in record file {record_file}. Skipping.")
                continue

            id = "_".join(custom_id.split("_")[:-1])
            dim = custom_id.split("_")[-1]
            if id not in results:
                results[id] = r[custom_id]
                results[id]["file_path"] = r[custom_id]["file_path"][L:]
                results[id]["source_path"] = r[custom_id]["source_path"][L:]
                results[id]["result"] = {}

            msg = result["response"]["body"]["choices"][0]["message"]["content"]
            input_token += result["response"]["body"]["usage"]["prompt_tokens"]
            output_token += result["response"]["body"]["usage"]["completion_tokens"]
            try:
                obj = json.loads(clear_json(msg))
                # json to dict
                obj = {k: v for k, v in obj.items()}
            except json.JSONDecodeError:
                print(f"Error decoding JSON for custom ID {custom_id} in file {results_file}. Skipping.")
                obj = {}
            results[id]["result"][dim] = obj

with open("output/batch_results.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(list(results.values()), ensure_ascii=False, indent=4))
